In [1]:
import pandas as pd
import numpy as np
import re

ks_df = pd.read_csv('kitchenstories_01112020.csv')
pd.set_option('display.max_columns', 100)
#ks_df.head(5)

### Cleaning lists / string of list in the scraped data

In [2]:
#CLEANING LISTS / STRING OF LIST IN THE SCRAPED DATA


#ingredient_quantity attribute being modified from string of values to list of float values
ingredient_quantity_fixed = []

for i in range(len(ks_df['ingredient_quantity'])):
    if ks_df['ingredient_quantity'][i][0] == '[':
        ingredient_quantity_fixed.append(ks_df['ingredient_quantity'][i].strip('][').split(', '))
    else:
        ingredient_quantity_fixed.append(ks_df['ingredient_quantity'][i].split(','))

[[float(i) for i in f] for f in ingredient_quantity_fixed]
ks_df['ingredient_quantity'] = ingredient_quantity_fixed


ks_df.ingredient_list = ks_df.ingredient_list.apply(lambda s: s.split(','))

ks_df.ingredient_unit = ks_df.ingredient_unit.apply(lambda s: s.split(','))


#Utensils attribute being modified from string to list of strings
utensils_fixed = []
for i in range(len(ks_df['utensils'])):
    if type(ks_df['utensils'][i]) == str:
        utensils_fixed.append(ks_df['utensils'][i].split(','))
    else:
        utensils_fixed.append(ks_df['utensils'][i])
        
ks_df['utensils'] = utensils_fixed

#### Adding an author_group attribute

In [ ]:
#AUTHOR GROUP 

#Checking unqiue values seen in the author_type column

print(ks_df.author_type.unique())


#Adding a grouping column for authors to seperate KS Team and other member types

author_group = []
for i in ks_df['author_type']:
    if i.find('Kitchen Stories') != -1:
        author_group.append('KS Team')
    else:
        author_group.append(i)
        
ks_df['author_group'] = author_group

print('\nAdded author_group with the following values:\n')
print(ks_df.author_group.unique())

#### Adding a total_time attribute

In [3]:
ks_df['total_time'] = ks_df['prep_time'] + ks_df['bake_time'] + ks_df['rest_time']

In [4]:
#Writing this version of the file to csv for all data manipulation purposes related to unique recipes

ks_df.to_csv(r'KS_Cleaned.csv', index = False)

### Dataframe with ingredients split from list
Creating a version of the dataframe with ingredients listed out in seperate rows, rather than lists
The recipe will then be copied over as many times as there are ingredients, but more analysis can be performed
with this dataframe

In [5]:
#Creating a copy to avoid manipulating the original dataframe
ks_df_ingredients = ks_df.copy()

def ingredient_info(column_name):
    dish_index = 0
    lis = []
    
    for data_type in column_name:
        lis.extend(map(lambda data: [dish_index, data], data_type))
        dish_index +=1
    
    return lis

ingredient = ingredient_info(ks_df_ingredients.ingredient_list)
# quantity = ingredient_info(ks_df_ingredients.ingredient_quantity)
# unit = ingredient_info(ks_df_ingredients.ingredient_unit)

ingredient_info = pd.DataFrame(ingredient, columns=['index', 'ingredient'])
# quantity = pd.DataFrame(quantity, columns=['index', 'quantity'])
# unit = pd.DataFrame(unit, columns=['index', 'unit'])

#Ensuring the number of rows match between the three attributes
# print(len(ingredient_info.index))
# print(len(quantity.index))
# print(len(unit.index))

# ingredient_info['ingredient_quantity'] = quantity.quantity
# ingredient_info['ingredient_unit'] = unit.unit


#Checking what the created dataframe looks like
print(ingredient_info.head(20))


#Dropping ingredient_list, ingredient_quantity, ingredient_unit 
ks_df_ingredients = ks_df_ingredients.drop('ingredient_list', axis = 1)


#Merging the two dataframes
ks_df_ingredients = pd.merge(ks_df_ingredients, ingredient_info, how = 'right', left_index = True, right_on = 'index')


#Additional cleaning of ingredients
rep_chars = 'cup|oz|\)|\(|\-|for|serving|garnish|shaking|bredding|drained|chopped|diced|frying|\
            ripe|cans|can|marinating|sprinkling|teaspoon|of|additional|or|coating|\/|msg|fresh|\
            diced|shaved|toasted|divided|optional|to|serve|taste|deep|frying|dusting|\d|\.'

ks_df_ingredients['ingredient'] = ks_df_ingredients['ingredient'].str.lower()
ks_df_ingredients['ingredient'] = ks_df_ingredients['ingredient'].str.replace(rep_chars,'')
ks_df_ingredients['ingredient'] = ks_df_ingredients['ingredient'].str.strip()
ks_df_ingredients['ingredient'].replace('', np.nan, inplace = True)
ks_df_ingredients.dropna(subset = ['ingredient'], inplace = True)
ks_df_ingredients['ingredient'] = ks_df_ingredients['ingredient'].str.replace(r' +',' ')


#Re-ordering columns to read dataframe easily
col_names = ['index', 'dish_name', 'dish_description', 'difficulty', 'rating', 'reviews_for_rating', 'image_count',
            'author', 'author_type', 'author_group', 'prep_time', 'bake_time', 'rest_time', 'total_steps', 'ingredient',
            'ingredient_quantity', 'ingredient_unit', 'calories', 'protein', 'protein_u', 'fat', 'fat_u', 'carb', 
             'carb_u', 'servings', 'utensils']

ks_df_ingredients = ks_df_ingredients.reindex(columns = col_names)
ks_df_ingredients.head(20)


    index                 ingredient
0       0            chicken breasts
1       0    Nando’s peri-peri sauce
2       0      smoked paprika powder
3       0                     garlic
4       0               ginger juice
5       0          mixed dried herbs
6       0                 lime juice
7       0                   couscous
8       0                feta cheese
9       0  jarred pitted black olive
10      0            cherry tomatoes
11      0            cucumber slices
12      0      full-fat plain yogurt
13      0                       milk
14      0                mixed herbs
15      0                     pepper
16      0                  olive oil
17      0                     butter
18      0               Sichuan salt
19      1      smoked bacon lardons 


,index,dish_name,dish_description,difficulty,rating,reviews_for_rating,image_count,author,author_type,author_group,prep_time,bake_time,rest_time,total_steps,ingredient,ingredient_quantity,ingredient_unit,calories,protein,protein_u,fat,fat_u,carb,carb_u,servings,utensils
0,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,chicken breasts,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
1,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,nando’s periperi sauce,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
2,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,smoked paprika powder,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
3,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,garlic,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
4,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,ginger juice,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
5,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,mixed dried herbs,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
6,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,lime juice,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
7,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,couscous,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
8,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,feta cheese,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"
9,0,Tender chicken skewers with couscous and feta dip,Perfect for a sunny Saturday lunch on the porch.,Easy,0.0,NaN,1,Palate & Plate,Community Member,NaN,35,0,30,8.0,jarred pitted black olive,"[3, 0.3333333333333333, 1.5, 2, 1, 1, 2, 1, 1,...","[cup, tsp, cloves, tbsp, tsp, tbsp, cup, cup, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[wooden skewer, aluminum foil]"


In [6]:
ks_df_ingredients.to_csv(r'KS_Ingredients_Cleaned.csv', index = False)

#### ingredient_quantity & ingredient_unit
I would've liked to also add ingredient_quantity and ingredient_unit columns outside of the list, same was as ingredient_list was generated, although it seems like there was an error when scraping data (or potentially in the list creation phase in Scrapy). The analysis below proves it:

In [7]:
test = ks_df.copy()
lis = []
lisq =[]
lisu =[]
for i in test.ingredient_list:
    lis.append(len(i))
for i in test.ingredient_quantity:
    lisq.append(len(i))
for i in test.ingredient_unit:
    lisu.append(len(i))

#Checking length of each list to ensure it captured all rows
print('Length of lists created:')
print(len(lis))
print(len(lisq))
print(len(lisu),'\n')


#Checking to see where ingredient_list and ingredient_quantity differ. ingredient_unit was created the same way
#as ingredient_quantity
i = 0
out = []

for x in lis: 
    if x != lisq[i]: 
        out.append(i) 
    i += 1

print(f'A total of {len(out)} rows differ between ingredient_list and ingredient_quantity / unit.\n')
print('Locations of where ingredient and quantity lists differ:\n', out,'\n')

Length of lists created:
606
606
606 

A total of 63 rows differ between ingredient_list and ingredient_quantity / unit.

Locations of where ingredient and quantity lists differ:
 [4, 24, 32, 60, 69, 79, 84, 101, 108, 112, 124, 130, 135, 136, 137, 139, 143, 173, 203, 211, 217, 218, 220, 240, 259, 263, 271, 272, 273, 278, 319, 320, 321, 325, 338, 345, 355, 371, 379, 411, 435, 444, 446, 481, 484, 490, 496, 508, 512, 516, 521, 527, 533, 535, 539, 547, 549, 551, 555, 556, 561, 568, 572] 

